## Framework / Setup

In [ ]:
!pip install mujoco
!pip install mujoco-py
!pip install numpy
!pip install scipy
!pip install mediapy
!git clone https://github.com/LSCodeRocket/Cartpole-Stabilization
!cd Cartpole-Stabilization; git pull

Cloning into 'Cartpole-Stabilization'...
remote: Enumerating objects: 202, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 202 (delta 38), reused 186 (delta 25), pack-reused 0
Receiving objects: 100% (202/202), 7.08 MiB | 16.11 MiB/s, done.
Resolving deltas: 100% (38/38), done.
Already up to date.


In [ ]:
# Set up GPU rendering.
from google.colab import files
import distutils.util
import os
import subprocess
import random
os.chdir("Cartpole-Stabilization/ROBOT_ARM/RobotArmMJCF/")
# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

# Check if installation was succesful.
try:
  print('Checking that the installation succeeded:')
  import mujoco as mj
  from mujoco.glfw import glfw
  mj.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')


Setting environment variable to use GPU rendering:
env: MUJOCO_GL=egl
Checking that the installation succeeded:
Installation successful.


In [ ]:
# Other imports and helper functions
import time
import itertools
import numpy as np
import scipy as sp
# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

from IPython.display import clear_output
clear_output()

### MuJoCoBase Class

In [ ]:
class MuJoCoBase():
    def __init__(self, xml_path):
        # For callback functions
        self.button_left = False
        self.button_middle = False
        self.button_right = False
        self.lastx = 0
        self.lasty = 0

        # MuJoCo data structures
        self.model = mj.MjModel.from_xml_path(xml_path)  # MuJoCo model
        self.data = mj.MjData(self.model)                # MuJoCo data
        self.cam = mj.MjvCamera()                        # Abstract camera
        self.opt = mj.MjvOption()                        # visualization options

        # initialize visualization data structures
        mj.mjv_defaultCamera(self.cam)
        mj.mjv_defaultOption(self.opt)
        # self.scene = mj.MjvScene(self.model, maxgeom=10000)
        # self.context = mj.MjrContext(
        #     self.model, mj.mjtFontScale.mjFONTSCALE_150.value)

    def simulate(self):
        pass

    def reset(self, *args, **kwargs):
        raise NotImplementedError

    def controller(self, *args, **kwargs):
        raise NotImplementedError

## Robotics Content

Just for reference

Numpy -> `np`

MuJoCo -> `mj`

Scipy -> `sp`

Matplotlib.pyplot -> `plt`

### Model


In [ ]:

mjcf_file = open('scene.xml', 'r')
xml = mjcf_file.read()
mjcf_file.close()


In [ ]:
model = mj.MjModel.from_xml_string(xml) # create model
data = mj.MjData(model) # get data

renderer = mj.Renderer(model)

duration = 3.8  # (seconds)
framerate = 20  # (Hz)

# Simulate and display video.
frames = []
mj.mj_resetData(model, data)  # Reset state and time.


In [ ]:
class RobotArm(MuJoCoBase):
  BASE_ROTATOR = {"ID" : 0, "RANGE" : [-2.9, 2.9]}
  LINK_1 = {"ID": 1, "RANGE": [-1.76, 1.76]}
  LINK_2 = {"ID": 2, "RANGE": [-2.9, 2.9]}
  LINK_3 = {"ID": 3, "RANGE": [-3.07, -0.0698]}
  LINK_4 = {"ID": 4, "RANGE": [-2.9, 2.9]}
  LINK_5 = {"ID": 5, "RANGE": [-0.0175, 3.75]}
  LINK_6 = {"ID": 6, "RANGE": [-2.9, 2.9]}
  END_EFFECTOR = {"ID": 7, "RANGE": [0, 255]}
  ACTUATOR_REFERENCE = [BASE_ROTATOR, LINK_1, LINK_2, LINK_3, LINK_4, LINK_5, LINK_6, END_EFFECTOR]

  def __init__(self, xml_path):
      super().__init__(xml_path)
      self.simend = 10.0


  def reset(self):
      pass

  def controller(self, model, data):
      if data.time < 2.5:
          data.ctrl = np.array([-0.29, 0.458, 0.232, -2.43, -0.174, 2.98, 0.811, 255])*((data.time)/2.5)
      elif data.time < 4:
          data.ctrl[-1] = 255 * ((4-data.time)/1.5 )
          self.initial_to_target = data.ctrl
      else:
          print("data.ctrl => ", data.ctrl)
          print("self.throw_target => ", self.throw_target)
          print("self.initial_to_target => ", self.initial_to_target)

          print((data.time))
          """
          data.ctrl = self.throw_target*((data.time-4)/3000) + (np.array(self.initial_to_target)) * (1-(data.time-4)/3000)
          data.ctrl[-1] = 0
          """
          data.ctrl += (self.throw_target-np.array(self.initial_to_target)) * (1/(300*(self.simend-4)))
  def simulate(self):
      renderer = mj.Renderer(model, 600, 800)
      framerate = 60  # (Hz)

      # Simulate and display video.
      frames = []
      mj.mj_resetData(model, data)  # Reset state and time.
      self.throw_target = np.random.rand(len(data.ctrl))
      print(self.throw_target)
      while data.time < self.simend:
        mj.mj_step(model, data)
        self.controller(model, data)
        if len(frames) < data.time * framerate:
          renderer.update_scene(data)
          pixels = renderer.render()
          frames.append(pixels)
      media.show_video(frames, fps=framerate)

In [ ]:
xml_path = "scene.xml"
model = mj.MjModel.from_xml_string(xml)
data = mj.MjData(model)
sim = RobotArm(xml_path)
sim.reset()
sim.simulate()


Streaming output truncated to the last 5000 lines.
data.ctrl =>  [ 0.458  0.343  0.491 -0.417  0.482  1.351  0.48   0.574]
self.throw_target =>  [0.914 0.273 0.648 0.808 0.882 0.361 0.279 0.922]
self.initial_to_target =>  [ 0.458  0.343  0.491 -0.417  0.482  1.351  0.48   0.574]
7.501999999999396
data.ctrl =>  [ 0.459  0.343  0.491 -0.416  0.483  1.351  0.48   0.574]
self.throw_target =>  [0.914 0.273 0.648 0.808 0.882 0.361 0.279 0.922]
self.initial_to_target =>  [ 0.459  0.343  0.491 -0.416  0.483  1.351  0.48   0.574]
7.503999999999396
data.ctrl =>  [ 0.459  0.343  0.491 -0.415  0.483  1.35   0.48   0.574]
self.throw_target =>  [0.914 0.273 0.648 0.808 0.882 0.361 0.279 0.922]
self.initial_to_target =>  [ 0.459  0.343  0.491 -0.415  0.483  1.35   0.48   0.574]
7.505999999999395
data.ctrl =>  [ 0.459  0.343  0.491 -0.415  0.483  1.35   0.48   0.574]
self.throw_target =>  [0.914 0.273 0.648 0.808 0.882 0.361 0.279 0.922]
self.initial_to_target =>  [ 0.459  0.343  0.491 -0.415  0.483  

This browser does not support the video tag.


## Failed Gradient Descent Code

In [ ]:
"""
model = mj.MjModel.from_xml_string(xml)
data = mj.MjData(model)

# Initial joint positions
qpos0 = np.array([0, 0.501792, -0.000125065, -2.28056, -0.000112028, 2.84613, 0.840131, 0.0249238, 0.0249238, 255])

# Set initial joint positions
data.qpos[:len(qpos0)] = qpos0
mj.mj_forward(model, data)

# Get target position of the end effector
target = data.body('link7').xpos.copy()
print("Target =>", target)

# Gradient Descent method
class GradientDescentIK:

    def __init__(self, model, data, step_size, tol, alpha, jacp, jacr):
        self.model = model
        self.data = data
        self.step_size = step_size
        self.tol = tol
        self.alpha = alpha
        self.jacp = jacp
        self.jacr = jacr

    def check_joint_limits(self, q):
        #Check if the joints are under or above their limits
        for i in range(len(q[0:-6])):
            q[i] = max(self.model.jnt_range[i][0], min(q[0:-6][i], self.model.jnt_range[i][1]))

    # Gradient Descent pseudocode implementation
    def calculate(self, goal, init_q, body_id):
        #Calculate the desired joint angles for the goal
        self.data.qpos[:len(init_q)] = init_q
        mj.mj_forward(self.model, self.data)
        current_pose = self.data.body(body_id).xpos
        error = np.subtract(goal, current_pose)

        while np.linalg.norm(error) >= self.tol:
            # Calculate Jacobian
            mj.mj_jacBodyCom(self.model, self.data, self.jacp, self.jacr, body_id)
            # Calculate gradient
            grad = self.alpha * self.jacp.T @ error
            # Compute next step
            self.data.qpos[:len(grad)] += self.step_size * grad
            # Check joint limits
            self.check_joint_limits(self.data.qpos)
            # Compute forward kinematics
            mj.mj_forward(self.model, self.data)
            # Calculate new error
            error = np.subtract(goal, self.data.body(body_id).xpos)
        return self.data.qpos.copy()

# Init variables
body_id = model.body('link7').id
jacp = np.zeros((3, model.nv))  # Translation Jacobian
jacr = np.zeros((3, model.nv))  # Rotational Jacobian
goal = np.array([-0.25, -0.25, 0.6])
step_size = 0.005
tol = 0.1
alpha = 0.5
init_q = np.zeros(model.nq)  # Initial joint angles

# Initialize IK solver
ik = GradientDescentIK(model, data, step_size, tol, alpha, jacp, jacr)

# Get desired point
mj.mj_resetDataKeyframe(model, data, 0)  # Reset qpos to initial value
ik.calculate(goal, init_q, body_id)  # Calculate the joint angles

result = data.qpos.copy()

# Plot results
print("Results")
data.qpos = qpos0
mj.mj_forward(model, data)

# Visualization setup
sim.reset()
# Print target and result positions
print("Target point =>", target)
print("Results => ", result)
print("Gradient Descent result =>", data.body('link7').xpos, "\n")

"""

'\nmodel = mj.MjModel.from_xml_string(xml)\ndata = mj.MjData(model)\n\n# Initial joint positions\nqpos0 = np.array([0, 0.501792, -0.000125065, -2.28056, -0.000112028, 2.84613, 0.840131, 0.0249238, 0.0249238, 255])\n\n# Set initial joint positions\ndata.qpos[:len(qpos0)] = qpos0\nmj.mj_forward(model, data)\n\n# Get target position of the end effector\ntarget = data.body(\'link7\').xpos.copy()\nprint("Target =>", target)\n\n# Gradient Descent method\nclass GradientDescentIK:\n\n    def __init__(self, model, data, step_size, tol, alpha, jacp, jacr):\n        self.model = model\n        self.data = data\n        self.step_size = step_size\n        self.tol = tol\n        self.alpha = alpha\n        self.jacp = jacp\n        self.jacr = jacr\n\n    def check_joint_limits(self, q):\n        #Check if the joints are under or above their limits\n        for i in range(len(q[0:-6])):\n            q[i] = max(self.model.jnt_range[i][0], min(q[0:-6][i], self.model.jnt_range[i][1]))\n\n    # Gradie